In [5]:
import logging,sys
def readFileUsingStreams(filename):
    '''
    readFileUsingStreams is used to check for the file format and read the files- regarless it is a data file or meta-data file.
    If the file format is supported, it returns the content of the File in seperate lines.
    If the  format of the file is not supported , the subroutine shall explicitly explain why it Failed.
    This function shall be used by other functions to read the file.
    '''
    lines=[]
    filename=filename # The file to be read
    extension=filename.split(".")[-1] #Checking the file extension
    delim=''
    if extension=='csv' or 'xls' or 'xlsx': # Based on the file type the delimiter shall be choosen
        delim=',' # Just for a prototype csv is being considered
    with open(filename,encoding="utf-8") as fhandler:
        try:
            for line in fhandler.readlines():
                lines.append(line.strip().split(delim))
                #print(lines)
            return(lines)
        except IOError as e:
                print ("I/O error({0}): {1}".format(e.errno, e.strerror))
                logging.basicConfig(filename="Error.log",level=2)
                logging.error(str(e))
        except Exception as E: #handle other exceptions such as attribute errors
            print ("Unexpected error:", sys.exc_info()[0])
            print('Error opening the file',E)
            logging.basicConfig(filename="Error.log",level=2)
            logging.error(str(E),sys.exc_info()[0]) # If this error has to be written to A file


In [6]:
from collections import OrderedDict,defaultdict
def parseMetaDateFile(metadatafile):
    '''
    parseMetaDateFile shall understand the meta data file and translate the definitions to implement on the data 
    file. So it shall be called by ParsefixedFile and it returns the meta-data definitions in a dictionary.
    '''
    metadatafile=metadatafile # meta data file to be translated
    extension=metadatafile.split(".")[-1] #Checking the file extension
    delimiter=''
    if extension=='csv' or 'xls' or 'xlsx': # Based on the file type the delimiter shall be choosen
        delimiter=',' # Just for a prototype csv is being considered
    lines=readFileUsingStreams(metadatafile)
    metadata=defaultdict(list)
    columnnames=[]                    
    for word in lines:
        inner_dict=dict()
        #print("words",word)
        inner_dict['length']=word[1]
        inner_dict['type']=word[2]
        metadata[word[0]]=inner_dict
        columnnames.append(word[0])
        
    #print(metadata)
    return metadata,columnnames

In [7]:
import csv,logging
class FormatError(Exception):   # U need to define ur own exception
    pass  # Pass is a Keyword which is a null operation-- when it is executed, nothing happens

def parseFixedFile(datafilename,metadatafilename):
    '''
    This function shall take the raw data file and meta data file and generate the final ouput file intended as
    per the meta data file. A csv file shall be created with the translated output.
    '''
    datafile=datafilename
    metadatafile=metadatafilename
    datalines=readFileUsingStreams(datafile)
    metadata,columns=parseMetaDateFile(metadatafile)
    lines=''
    output=[]
    for line in datalines:
        start=0
        lineoutput=[]
        line=''.join(line)
        for item in metadata:
            getType=metadata[item]['type'] #implementing the typecheck for date alone.Can be implemented for other
            # And remote cases aswell
            if getType=="date":
                if len(line[start:start+int(metadata[item]['length'])].strip())==int(metadata[item]['length']):
                    date_res=line[start:start+int(metadata[item]['length'])].strip().split('-')
                    lineoutput.append("-".join(date_res[::-1]))
                else:
                    raise FormatError("IP address Format is wrong") # In this case it will create its own exception
                    
            else:
                lineoutput.append(line[start:start+int(metadata[item]['length'])].strip())
            start=start+int(metadata[item]['length'])
        output.append(lineoutput)
   # print(output)
    
    with open('result.csv', 'a',encoding='utf-8') as csvFile:
        writer = csv.writer(csvFile)
        try:
            writer.writerow(columns)
            for row in output:
                writer.writerow(row)
        except Exception as E: #handle other exceptions such as attribute errors
            raise Exception('Exception: {}'.format(str(E).decode('utf-8')).encode('utf-8'))
            
            logging.basicConfig(filename="Encoding.log",level=2)
            logging.error(str(E),sys.exc_info()[0])
    csvFile.close()
    return ('Output file with name result.csv succesfuly created in the same location')

In [8]:
# Sample output
parseFixedFile(datafilename='abc.log',metadatafilename='meta_datafile.csv')

'Output file with name result.csv succesfuly created in the same location'